Analysis of FairSD vs Fair flat10_zec runs

In [1]:
import os
import numpy as np
# import matplotlib.pyplot as pl
import matplotlib.pyplot as plt
import pandas as pd
import pooch
import seaborn as sns
from tqdm.auto import tqdm
import xarray as xr
import numpy as np
import scipy.signal
import scipy
from IPython.display import display, Math

constrained_by_co20909_03152024.npy
constrained_by_co2_1639_03152024.npy
constrained_by_co22336_03142024.npy
constrained_by_gr0909_03152024.npy
constrained_by_gr_1639_03152024.npy
constrained_by_gr2336_03142024.npy
constrained_by_T0909_03152024.npy
constrained_by_T_1639_03152024.npy
constrained_by_T_2336_03142024.npy
constrained_gems0910_03152024.npy
constrained_gems_1639_03152024.npy
constrained_gems2336_03142024.npy
fair_local/
fairSD_analysis.ipynb
fairSD_run.ipynb
gems_constrained_config_ens_1432_03142024.npy
landc_zec_response.png
